In [1]:

import requests
import json
import pandas as pd
import sys


In [ ]:

def get_json_data(url, timeout=10, max_retries=3, show_stats=True):
    """Downloads a JSON file from a URL with robust error handling and data summary.

    Args:
        url: The URL of the JSON file.
        timeout: The maximum number of seconds to wait for a server response.
        max_retries: The maximum number of retries to attempt download in case of errors.
        show_stats: Boolean to indicate whether to show descriptive statistics.

    Returns:
        A dictionary containing the JSON data, or None if an error occurs after retries.
    """
    data = None

    for attempt in range(1, max_retries + 1):
        try:
            response = requests.get(url, timeout=timeout)
            response.raise_for_status()

            # Parse the response content as JSON
            data = response.json()

            # Download successful, break out of retry loop
            break

        except (requests.exceptions.RequestException, TimeoutError) as e:
            print(f"Error downloading JSON from {url} (attempt {attempt}/{max_retries}): {e}")
            if attempt == max_retries:
                print(f"Failed to download JSON after {max_retries} attempts.")
                return None

        except json.JSONDecodeError as e:
            print(f"Error parsing JSON data: {e}")
            return None

    # Print the downloaded data dictionary
    print("\nDownloaded Data:")
    print(json.dumps(data, indent=4, ensure_ascii=False))

    # Calculate and print the size of the JSON object
    json_size = sys.getsizeof(data)
    print(f"\nSize of the JSON object: {json_size} bytes")

    # Data Description using pandas (assuming data is a dictionary)
    if isinstance(data, dict):
        print("\nData Description:")

        try:
            # Convert data to a pandas DataFrame (if possible)
            df = pd.DataFrame([data])  # Wrap the dictionary in a list to create a single-row DataFrame

            # Print total rows
            print(f"\nTotal rows: {len(df)}")

            # Explore some basic information about the data
            print("\nBasic Info:")
            print(df.info())

            # Print column names and data types
            print("\nColumn Names and Data Types:")
            print(df.dtypes)

            # Print column headers
            print("\nColumn Headers:")
            print(df.columns.tolist())

            # Check for missing values
            print("\nMissing Values:")
            print(df.isnull().sum())

            if show_stats:
                # Descriptive statistics for numerical columns
                print("\nDescriptive Statistics:")
                print(df.describe(include='all'))

        except (ValueError, KeyError) as e:
            print(f"Error converting data to pandas DataFrame: {e}")

    return data


In [ ]:
url = "https://servicios.ine.es/wstempus/js/ES/SERIE/CENSO1568062"
data = get_json_data(url, timeout=10, max_retries=3)


In [ ]:

if data:
    # You can now process the data further (e.g., perform more in-depth EDA)
    pass
else:
    print("Failed to download JSON data.")
